In [ ]:
# process
# load excel sheets with data
# set parameters (time of bins, frequency bands, designate one as 'tremor frequency')
# calculate averages for each animal
# compile averages for each animal into sheet
# calculate TPR for each bin 
# output data in exact format for prism
    # parameterize group and animal numbers as input
    # organize data as columns == animals, rows == time bins, columns organized by group

In [3]:
import pandas as pd
import numpy as np

In [4]:
fn = "/Users/rebeccakrall/Data/Tremor Project/TPR data all animals.xlsx"
excel_tables = pd.read_excel(fn, sheet_name=None)


In [5]:
animal_key = pd.read_excel("/Users/rebeccakrall/Data/Tremor Project/Animal_group_key.xlsx", index_col = 0)

In [6]:
animals = [a for a in excel_tables.keys() if a != 'Summary']

In [7]:
sheet = excel_tables[animals[0]]

In [55]:
data = sheet.iloc[0:256, :].set_index('f (Hz)').iloc[:, 2:]
new_cols = np.linspace(0, 10.24 * (sheet.shape[1]-1), sheet.shape[1])/60

data.columns = new_cols[2:-1]

In [50]:
parameters = {'hz': [2,8,14,25], 'bin_time': 5, 'col_time':10.24, 'exp_length' : 60, 'tremor_freq': 1}

In [53]:
def process_excel_sheet(sheet, parameters):
    
    data = sheet.iloc[0:256, :].set_index('f (Hz)').iloc[:, 2:]
    new_cols = np.linspace(0, parameters['col_time'] * (sheet.shape[1]-1), sheet.shape[1])/60
    data.columns = new_cols[2:-1]

    col_bins = np.linspace(0, parameters['exp_length'] , int(parameters['exp_length'] / parameters['bin_time']) +1)
    frequency_bins  = pd.cut(data.index, bins = parameters['hz'])
    time_bins = pd.cut(data.columns, bins = col_bins) 
    a = data.groupby([frequency_bins], observed = False).mean().T.groupby([time_bins], observed = False).mean() * 1000

    non_target_columns = [col for col in range(len(a.columns)) if col != parameters['tremor_freq']]
    tpr = 2 * a.iloc[:, parameters['tremor_freq']] / a.iloc[:, non_target_columns].sum(axis=1)
    # tpr = 2 * a.iloc[:,parameters['tremor_freq']] / (a.iloc[:,0]+ a.iloc[:,2])

    return a, tpr
   

In [54]:
def compile_excel_sheets(excel_tables, animals, animal_key, parameters):
    columns = pd.MultiIndex(levels=[[], []], codes=[[], []], names=[ 'Group', 'Subject'])
    tprs = pd.DataFrame(columns = columns)

    columns2 = pd.MultiIndex(levels=[[], [], []], codes=[[], [],[]], names=[ 'Group', 'Subject', 'Frequency'])
    freqs = pd.DataFrame(columns=columns2)

    all_freq = []
    for an in animals:
        if int(an) in animal_key.index:
            group = animal_key.loc[int(an), 'GROUP']
            column_index = (group, an)

            freq, trp = process_excel_sheet(excel_tables[an], parameters)
            
            multi_index = pd.MultiIndex.from_arrays([[group] * 3, [an] * 3, freq.columns], names=['Group', 'Subject', 'Frequency'])
            freq.columns = multi_index
            # freqs[multi_index] = freq

            all_freq.append(freq)
            tprs[column_index] = trp

    freqs = pd.concat(all_freq, axis = 1)

    return freqs, tprs

In [51]:
non_target_columns = [col for col in range(len(a.columns)) if col != parameters['tremor_freq']]

In [56]:
a, tpr = process_excel_sheet(excel_tables['11'], parameters)

In [57]:
a

,"(2, 8]","(8, 14]","(14, 25]"
"(0.0, 5.0]",194.24824,71.090213,14.772575
"(5.0, 10.0]",67.937679,30.983979,6.499227
"(10.0, 15.0]",11.835019,22.129304,3.523532
"(15.0, 20.0]",0.538345,1.112722,0.634037
"(20.0, 25.0]",74.081411,41.556834,7.778708
"(25.0, 30.0]",91.917646,52.9152,9.977079
"(30.0, 35.0]",7.083173,8.373174,3.407546
"(35.0, 40.0]",15.959205,19.969055,4.330484
"(40.0, 45.0]",0.133972,0.626267,0.554065
"(45.0, 50.0]",0.283087,0.904476,0.776319


In [58]:
tpr

(0.0, 5.0]      0.680221
(5.0, 10.0]      0.83249
(10.0, 15.0]    2.881692
(15.0, 20.0]    1.898223
(20.0, 25.0]    1.015313
(25.0, 30.0]    1.038625
(30.0, 35.0]    1.596301
(35.0, 40.0]    1.968394
(40.0, 45.0]    1.820446
(45.0, 50.0]    1.707514
(50.0, 55.0]    1.705869
(55.0, 60.0]    0.622725
dtype: object